# Dataset transforms

We often need to transform large amounts of data into compatible/usable formats. PyTorch has implemented a large number of transforms that we can use out of the box. In this example we will work on the MNIST dataset and use the `ToTensor` transform which will convert images or numpy arrays to tensors. <br>

Transforms can be applied to PIL images, tensors, ndarrays, or custom data during creation of the Dataset class. <br>
Complete list of built-in transforms can be found <a href="https://pytorch.org/vision/stable/transforms.html">here.</a>
- On Images
```
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
```
- On Tensors
```
LinearTransformation, Normalize, RandomErasing
```
- Conversion
```
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
```
- Generic
```
Use Lambda 
```
- Custom
```
Write own class
```
- Compose multiple Transforms
```
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
```

In [1]:
# Imports
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

In [3]:
# Copying and extending the custom dataset from the previous notebook
class WineDataset(Dataset):  # Extends the Dataset module
    def __init__(self, transform=None):  # To support transforms, None by default
        # Data loading
        xy = np.loadtxt('data/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        # Split and convert
        self.x = torch.from_numpy(xy[:, 1:])  # All the samples except first column
        self.y = torch.from_numpy(xy[:, [0]])  # n_samples, 1
        self.num_samples = xy.shape[0]  # Number of samples

        self.transform = transform

    def __getitem__(self, index):  # Will allow for indexing later
        # dataset[0] would be valid due to this method
        sample = self.x[index], self.y[index]
        # Apply a transform if available:
        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):  # This allows us to get the length of the dataset
        return self.num_samples

In [10]:
# Now that we've modified our dataset to use transforms, we can use ready made ones
# But for now we're gonna implement one on our own:
class ToTensor:
    def __call__(self, sample):  # Callable object
        # Unpack the sample
        inputs, labels = sample
        return torch.from_numpy(np.asarray(inputs)), torch.from_numpy(np.asarray(labels))


In [14]:
dataset = WineDataset(transform=ToTensor())

first_data = dataset[0]
test_features, test_labels = first_data
print(f'Test features type: {type(test_features)}, Test labels: {type(test_labels)}')
print(f'Test features: {test_features}')


Test features type: <class 'torch.Tensor'>, Test labels: <class 'torch.Tensor'>
Test features: tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])


In [15]:
# Multiplication transform class implementation
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, labels = sample
        inputs *= self.factor
        return inputs, labels

In [16]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])  # Takes a list of transforms

dataset = WineDataset(transform=composed)

first_data = dataset[0]
test_features, test_labels = first_data
print(f'Test features type: {type(test_features)}, Test labels: {type(test_labels)}')
print(f'Test features: {test_features}')

Test features type: <class 'torch.Tensor'>, Test labels: <class 'torch.Tensor'>
Test features: tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
